In [1]:
import numpy as np 
import pandas as pd
import datetime
import calendar
import matplotlib.pyplot as plt
import seaborn as sns
import re # regex
from scipy.misc import imread
#from wordcloud import WordCloud, STOPWORDS # Wordcloud: pip install wordcloud

%matplotlib inline

In [2]:
postulantes_educacion = pd.read_csv('../data/1_postulantes_educacion_train.csv')
postulantes_genero_edad = pd.read_csv('../data/2_postulantes_genero_edad_train.csv')
vistas = pd.read_csv('../data/3_vistas_train.csv')
postulaciones = pd.read_csv('../data/4_postulaciones_train.csv')
avisos_online = pd.read_csv('../data/5_avisos_online_train.csv')
avisos_detalle = pd.read_csv('../data/6_avisos_detalle_train.csv')

# Juntamos de los postulantes la informacion sobre educacion, genero, y edad

### Obtenemos el maximo nivel de educacion para cada postulante

In [3]:
postulantes_educacion.head(2)

,idpostulante,nombre_estudio,estado_estudio
0,8BkL,Universitario,En Curso
1,1d2B,Universitario,En Curso


In [4]:
len(postulantes_educacion)

568612

In [8]:
print('Existen datos de educacion de', len(postulantes_educacion.drop_duplicates(['idpostulante'], keep='last', inplace=False)), 'postulantes')

Existen datos de educacion de 366395 postulantes


In [9]:
def cuantificar_estudios(row):
    if (row['nombre_estudio'] == 'Doctorado'):
        row['nombre_estudio'] = 7
    if (row['nombre_estudio'] == 'Master'):    
        row['nombre_estudio'] = 6
    if (row['nombre_estudio'] == 'Posgrado'):    
        row['nombre_estudio'] = 5
    if (row['nombre_estudio'] == 'Universitario'):    
        row['nombre_estudio'] = 4
    if (row['nombre_estudio'] == 'Terciario/Técnico'):    
        row['nombre_estudio'] = 3
    if (row['nombre_estudio'] == 'Secundario'):    
        row['nombre_estudio'] = 2
    if (row['nombre_estudio'] == 'Otro'):    
        row['nombre_estudio'] = 1    
        
    if (row['estado_estudio'] == 'Graduado'):
        row['estado_estudio'] = 3
    if (row['estado_estudio'] == 'En Curso'):    
        row['estado_estudio'] = 2
    if (row['estado_estudio'] == 'Abandonado'):    
        row['estado_estudio'] = 1
        
    return row    

def descuantificar_estudios(row):
    if (row['nombre_estudio'] == 7):
        row['nombre_estudio'] = 'Doctorado'
    if (row['nombre_estudio'] == 6):    
        row['nombre_estudio'] = 'Master'
    if (row['nombre_estudio'] == 5):    
        row['nombre_estudio'] = 'Posgrado'
    if (row['nombre_estudio'] == 4):    
        row['nombre_estudio'] = 'Universitario'
    if (row['nombre_estudio'] == 3):    
        row['nombre_estudio'] = 'Terciario/Técnico'
    if (row['nombre_estudio'] == 2):    
        row['nombre_estudio'] = 'Secundario'
    if (row['nombre_estudio'] == 1):    
        row['nombre_estudio'] = 'Otro'    
        
    if (row['estado_estudio'] == 3):
        row['estado_estudio'] = 'Graduado'
    if (row['estado_estudio'] == 2):    
        row['estado_estudio'] = 'En Curso'
    if (row['estado_estudio'] == 1):    
        row['estado_estudio'] = 'Abandonado'
        
    return row    

In [10]:
postulantes_educacion = postulantes_educacion.apply(lambda row: cuantificar_estudios(row), axis=1)

In [11]:
# Obtenemos max de la columna nombre
idx = postulantes_educacion.groupby(['idpostulante'])['nombre_estudio'].transform(max) == postulantes_educacion['nombre_estudio']

In [12]:
# Sobre los anteriores, obtenemos los max de la columna estado
idx_postulantes_educacion_max = postulantes_educacion[idx].groupby(['idpostulante'])['estado_estudio'].transform(max) == postulantes_educacion[idx]['estado_estudio']

In [13]:
postulantes_educacion_max = postulantes_educacion[idx][idx_postulantes_educacion_max]
len(postulantes_educacion_max)

366395

In [14]:
# Verificamos, por ejemplo para este postulante debe traer el maximo nombre y estado
postulantes_educacion[postulantes_educacion['idpostulante'] == '1QLLO'] 

,idpostulante,nombre_estudio,estado_estudio
5,1QLLO,4,2
290,1QLLO,4,3


In [15]:
# Efectivamente lo hace
postulantes_educacion_max[postulantes_educacion_max['idpostulante'] == '1QLLO'] 

,idpostulante,nombre_estudio,estado_estudio
290,1QLLO,4,3


In [16]:
postulantes_educacion_max = postulantes_educacion_max.apply(lambda row: descuantificar_estudios(row), axis=1)

In [17]:
postulantes_educacion_max.head(2)

,idpostulante,nombre_estudio,estado_estudio
1,1d2B,Universitario,En Curso
3,6M3jr,Universitario,En Curso


### Join con genero y edad

In [18]:
len(postulantes_genero_edad)

408146

In [19]:
len(postulantes_genero_edad.drop_duplicates(['idpostulante'], keep='last', inplace=False))

408146

In [20]:
postulantes = pd.merge(postulantes_educacion_max, postulantes_genero_edad, on='idpostulante', how='inner')

In [21]:
len(postulantes)

366395

In [22]:
postulantes.head(2)

,idpostulante,nombre_estudio,estado_estudio,fechanacimiento,sexo
0,1d2B,Universitario,En Curso,1976-02-28,MASC
1,6M3jr,Universitario,En Curso,1975-11-03,MASC


### Join con vistas y postulaciones

In [23]:
print(len(vistas))
print(len(postulaciones))

6682591
6604534


In [24]:
#vistas.rename(columns={'idAviso':'idaviso','timestamp':'fechavista'}, inplace=True)
vistas.head(1)

,idaviso,fechavista,idpostulante
0,1111780242,2018-02-23T13:38:13.187-0500,YjVJQ6Z


In [25]:
postulaciones.head(1)

,idaviso,idpostulante,fechapostulacion
0,1112257047,NM5M,2018-01-15 16:22:34


In [26]:
vistas_postulaciones = pd.concat([vistas, postulaciones])

In [27]:
print(len(vistas_postulaciones))

13287125


In [28]:
vistas_postulaciones.to_csv('../data/vistas_postulaciones.csv', index=False, sep=',', encoding='utf-8')

In [29]:
postulantes.head(1)

,idpostulante,nombre_estudio,estado_estudio,fechanacimiento,sexo
0,1d2B,Universitario,En Curso,1976-02-28,MASC


In [30]:
vistas_postulaciones_postulantes = pd.merge(vistas_postulaciones, postulantes, on='idpostulante', how='outer')

In [31]:
print(len(vistas_postulaciones_postulantes))

13300822


In [32]:
vistas_postulaciones_postulantes.tail(2)

,fechapostulacion,fechavista,idaviso,idpostulante,nombre_estudio,estado_estudio,fechanacimiento,sexo
13300820,NaN,NaN,NaN,xkvqer0,Secundario,Abandonado,1972-12-05,FEM
13300821,NaN,NaN,NaN,rm04Rzd,Terciario/Técnico,Abandonado,1976-04-13,MASC


In [33]:
vistas_postulaciones_postulantes.to_csv('../data/postulantes_vistas_postulaciones.csv', index=False, sep=',', encoding='utf-8')

In [34]:
vistas_postulaciones_postulantes = pd.read_csv('../data/postulantes_vistas_postulaciones.csv')

C:\Users\tbotalla\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [35]:
vistas_postulaciones_postulantes.head(1)

,fechapostulacion,fechavista,idaviso,idpostulante,nombre_estudio,estado_estudio,fechanacimiento,sexo
0,NaN,2018-02-23T13:38:13.187-0500,1.111780e+09,YjVJQ6Z,Universitario,Abandonado,1991-11-16,MASC


### Limpieza de avisos_detalle

In [36]:
avisos_detalle.head(1)

,idaviso,titulo,descripcion,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,17903700,Enfermeras,Solicitamos para importante cadena de farmacia...,Full-time,Senior / Semi-Senior,Salud,Farmacias Central Oeste


### Merge con avisos_detalle

In [49]:
vistas_postulaciones_postulantes_avisos = pd.merge(vistas_postulaciones_postulantes, avisos_detalle, on='idaviso', how='left')

In [50]:
vistas_postulaciones_postulantes_avisos.to_csv('../data/vistas_postulaciones_postulantes_avisos.csv', index=False, sep=',', encoding='utf-8')

In [51]:
vistas_postulaciones_postulantes_avisos.head(1)

,fechapostulacion,fechavista,idaviso,idpostulante,nombre,estado,fechanacimiento,sexo,idpais,titulo,descripcion,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,NaN,2018-02-23T13:38:13.187-0500,1.11178e+09,YjVJQ6Z,Universitario,Abandonado,1991-11-16,MASC,1.0,"OPERARIO PARA DEPOSITO, CONFECCION DE PEDIDOS,...",<p>SE NECESITA OPERIARIO PARA DEPOSITO CONFECC...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Programación de producción,dm


In [52]:
vistas_postulaciones_postulantes_avisos.columns

Index(['fechapostulacion', 'fechavista', 'idaviso', 'idpostulante', 'nombre',
       'estado', 'fechanacimiento', 'sexo', 'idpais', 'titulo', 'descripcion',
       'nombre_zona', 'ciudad', 'mapacalle', 'tipo_de_trabajo',
       'nivel_laboral', 'nombre_area', 'denominacion_empresa'],
      dtype='object')

In [53]:
vistas_postulaciones_postulantes_avisos.memory_usage(index=False)

fechapostulacion        109940056
fechavista              109940056
idaviso                 109940056
idpostulante            109940056
nombre                  109940056
estado                  109940056
fechanacimiento         109940056
sexo                    109940056
idpais                  109940056
titulo                  109940056
descripcion             109940056
nombre_zona             109940056
ciudad                  109940056
mapacalle               109940056
tipo_de_trabajo         109940056
nivel_laboral           109940056
nombre_area             109940056
denominacion_empresa    109940056
dtype: int64

In [56]:
vistas_postulaciones_postulantes_avisos.to_csv('../data/vistas_postulaciones_postulantes_avisos.csv', index=False, sep=',', encoding='utf-8')

In [57]:
vistas_postulaciones_postulantes_avisos.columns

Index(['fechapostulacion', 'fechavista', 'idaviso', 'idpostulante',
       'nivel_educacion', 'estado_educacion', 'fechanacimiento', 'sexo',
       'titulo', 'descripcion', 'tipo_de_trabajo', 'nivel_laboral',
       'nombre_area', 'denominacion_empresa'],
      dtype='object')